# First notebook

In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(engines="mpi", n=2)
rc = cluster.start_and_connect_sync()


Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.41s/engine]


In [2]:
%%px
from mpi4py import MPI


print(f"{MPI.COMM_WORLD.rank=} {MPI.COMM_WORLD.size=}")


[stdout:1] MPI.COMM_WORLD.rank=1 MPI.COMM_WORLD.size=2


[stdout:0] MPI.COMM_WORLD.rank=0 MPI.COMM_WORLD.size=2


## Scatter and gather

In [3]:
%%px
# Based on: https://medium.com/@mathcube7/parallel-computing-in-python-c55c87c36611

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print("Hello, World! I am process", rank, "of", size, "on", MPI.Get_processor_name())
if rank == 0:
    data = [(i+1)**2 for i in range(size)]
    print("")
    print("I am the root process", rank, "and I will send (scatter) one number to each of the processes, then they are going to multiply it by 2 and send them back to me (gather).")
    print("These are the numbers I will send to the other", size, "process(es):", data)
    print("")
else:
    data = None

# Scattering data to other processes from process 0
scattered_data = comm.scatter(data, root=0)

# Each process will multiply the data they received
scattered_data = scattered_data * 2

# Gathering data in process 0
gathered_data = comm.gather(scattered_data, root=0)
if rank == 0:
    print("This is the resulting list of numbers:", gathered_data)

[stdout:0] Hello, World! I am process 0 of 2 on sidneysimula

I am the root process 0 and I will send (scatter) one number to each of the processes, then they are going to multiply it by 2 and send them back to me (gather).
These are the numbers I will send to the other 2 process(es): [1, 4]

This is the resulting list of numbers: [2, 8]


[stdout:1] Hello, World! I am process 1 of 2 on sidneysimula
